In [1]:
import os
import numpy as np
import pandas as pd
from time import time
from tqdm import tqdm
import json
import chromadb
from chromadb.config import Settings

from src.datasets.caltech256 import Caltech256DataModule
from src.featuring.rgb_histogram import RGBHistogram
from src.metrics import average_precision

# Tạo thư mục output
os.makedirs('out', exist_ok=True)

print("✅ Imports hoàn tất!")


✅ Imports hoàn tất!


In [2]:
# ======================== CONFIGURATIONS ========================
CONFIGS = [
    # (n_bin, h_type, metric)
    (4, 'global', 'cosine'),
    (4, 'global', 'l2'),
    (4, 'region', 'cosine'), 
    (4, 'region', 'l2'),
    (8, 'global', 'cosine'),
    (8, 'global', 'l2'),
    (8, 'region', 'cosine'),
    (8, 'region', 'l2'),
]

# Cấu hình evaluation
TEST_SIZE = 5956     # Số lượng test samples để evaluation
N_SLICE = 3          # Số slice cho region histogram
MAP_K_VALUES = [1, 5, 10]  # Top-k cho mAP evaluation
RECALL_K_VALUES = [10, 100, 1000]  # Top-k cho Recall evaluation

# ChromaDB settings
CHROMA_DIR = "chroma_storage"
COLLECTION_PREFIX = "rgb_hist"

print(f"📋 Tổng cấu hình để test: {len(CONFIGS)}")
print(f"📊 Test size: {TEST_SIZE}")
print(f"💾 ChromaDB storage: {CHROMA_DIR}")
print(f"🎯 mAP k-values: {MAP_K_VALUES}")
print(f"🎯 Recall k-values: {RECALL_K_VALUES}")


📋 Tổng cấu hình để test: 8
📊 Test size: 5956
💾 ChromaDB storage: chroma_storage
🎯 mAP k-values: [1, 5, 10]
🎯 Recall k-values: [10, 100, 1000]


In [3]:
# Setup dataset
dataset_root = os.path.abspath('data/caltech-256/256_ObjectCategories')
print(f"📂 Dataset path: {dataset_root}")
print(f"📂 Exists: {os.path.exists(dataset_root)}")

if os.path.exists(dataset_root):
    data_module = Caltech256DataModule(batch_size=32, root=dataset_root)
    data_module.setup()
    train_loader = data_module.train_dataloader()
    test_loader = data_module.test_dataloader()
    
    train_size = len(train_loader.dataset)
    test_size = len(test_loader.dataset)
    total_size = train_size + test_size
    
    print(f"✅ Dataset loaded thành công!")
    print(f"📊 Training: {train_size} samples ({train_size/total_size*100:.1f}%)")
    print(f"📊 Testing: {test_size} samples ({test_size/total_size*100:.1f}%)")
    print(f"📊 Total: {total_size} samples")
else:
    print("❌ Dataset không tìm thấy!")
    print("Vui lòng đảm bảo dataset ở đường dẫn đúng.")


📂 Dataset path: D:\AI\Food-CBIR\src\evaluation\data\caltech-256\256_ObjectCategories
📂 Exists: True
📂 Found 256 valid categories
📊 Loaded 29780 total images from 256 classes
📋 Train: 23824 images
📂 Found 256 valid categories
📊 Loaded 29780 total images from 256 classes
📋 Test: 5956 images
Train: 23824, Test: 5956
✅ Dataset loaded thành công!
📊 Training: 23824 samples (80.0%)
📊 Testing: 5956 samples (20.0%)
📊 Total: 29780 samples


In [4]:
# Setup GPU/CPU device
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Device: {device}")

if device.type == "cuda":
    print(f"📱 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"🔥 CUDA Version: {torch.version.cuda}")
    # Clear any existing GPU cache
    torch.cuda.empty_cache()
else:
    print("💻 Using CPU (GPU not available)")


🚀 Device: cuda
📱 GPU: NVIDIA GeForce RTX 4060 Laptop GPU
💾 GPU Memory: 8.0 GB
🔥 CUDA Version: 12.1


In [5]:
# Setup ChromaDB
os.makedirs(CHROMA_DIR, exist_ok=True)

chroma_client = chromadb.PersistentClient(
    path=CHROMA_DIR,
    settings=Settings(
        anonymized_telemetry=False,
        allow_reset=True
    )
)

print(f"✅ ChromaDB initialized tại: {CHROMA_DIR}")
print(f"📊 Existing collections: {len(chroma_client.list_collections())}")


✅ ChromaDB initialized tại: chroma_storage
📊 Existing collections: 8


In [6]:
def get_collection_name(n_bin, h_type, metric):
    """Generate collection name for configuration"""
    return f"{COLLECTION_PREFIX}_{n_bin}bin_{h_type}_{metric}"

def extract_and_store_features(n_bin, h_type, metric, force_reindex=False):
    """Extract features và store vào ChromaDB"""
    collection_name = get_collection_name(n_bin, h_type, metric)
    
    # Kiểm tra xem collection đã tồn tại chưa
    try:
        collection = chroma_client.get_collection(collection_name)
        count = collection.count()
        if count > 0 and not force_reindex:
            print(f"📂 Sử dụng collection có sẵn: {collection_name} ({count} vectors)")
            return collection, count
    except:
        pass  # Collection chưa tồn tại
    
    print(f"🔧 Tạo collection mới: {collection_name}")
    print(f"   Cấu hình: n_bin={n_bin}, h_type={h_type}, metric={metric}")
    
    # Xóa collection cũ nếu có
    try:
        chroma_client.delete_collection(collection_name)
    except:
        pass
    
    # Tạo collection mới
    chroma_metric = "cosine" if metric == "cosine" else "l2"
    collection = chroma_client.create_collection(
        name=collection_name,
        metadata={"n_bin": n_bin, "h_type": h_type, "metric": metric},
        embedding_function=None
    )
    
    # Extract features
    feature_extractor = RGBHistogram(n_bin=n_bin, h_type=h_type, n_slice=N_SLICE)
    
    print("📊 Extracting features từ training set...")
    stored_count = 0
    train_labels_map = {}
    
    start_time = time()
    
    for batch_idx, (images, labels, _) in enumerate(tqdm(train_loader, desc="Indexing")):
        # Move to GPU if available
        if device.type == "cuda":
            images = images.to(device)
        
        # Convert to numpy
        images_np = (images.cpu().numpy().transpose(0, 2, 3, 1) * 255).astype(np.uint8)
        
        # Extract features cho batch
        batch_features = []
        batch_ids = []
        batch_metadatas = []
        
        for i, (img, label) in enumerate(zip(images_np, labels)):
            feature = feature_extractor(img)
            
            vector_id = f"train_{stored_count + i}"
            
            batch_features.append(feature.tolist())
            batch_ids.append(vector_id)
            batch_metadatas.append({
                "type": "train",
                "label": int(label),
                "index": stored_count + i
            })
            
            train_labels_map[vector_id] = int(label)
        
        # Lưu batch vào ChromaDB
        if batch_features:
            collection.add(
                embeddings=batch_features,
                ids=batch_ids,
                metadatas=batch_metadatas
            )
        
        stored_count += len(batch_features)
    
    indexing_time = time() - start_time
    
    # Lưu labels mapping
    try:
        labels_json = json.dumps(train_labels_map)
        collection.add(
            embeddings=[[0.0] * len(batch_features[0])],
            ids=["_labels_map_"],
            metadatas=[{"type": "labels_map"}],
            documents=[labels_json]
        )
    except Exception as e:
        print(f"⚠️  Không thể lưu labels mapping: {e}")
    
    print(f"✅ Hoàn thành indexing: {stored_count} vectors trong {indexing_time:.2f}s")
    print(f"⚡ Tốc độ: {stored_count/indexing_time:.1f} vectors/s")
    
    return collection, stored_count

print("📋 Hàm indexing đã sẵn sàng!")


📋 Hàm indexing đã sẵn sàng!


In [14]:
# CHẠY INDEXING CHO TẤT CẢ CẤU HÌNH
print(f"🚀 Bắt đầu indexing cho {len(CONFIGS)} cấu hình...")
print(f"📊 Sẽ index {train_size} training samples cho mỗi cấu hình")
print("⏳ Quá trình này có thể mất vài phút...")

indexing_results = []

for i, (n_bin, h_type, metric) in enumerate(CONFIGS, 1):
    print(f"\n{'='*60}")
    print(f"Indexing {i}/{len(CONFIGS)}: {n_bin}bin_{h_type}_{metric}")
    print(f"{'='*60}")
    
    try:
        start_time = time()
        collection, count = extract_and_store_features(n_bin, h_type, metric)
        end_time = time()
        
        result = {
            'config': f"{n_bin}bin_{h_type}_{metric}",
            'n_bin': n_bin,
            'h_type': h_type,
            'metric': metric,
            'vectors_count': count,
            'indexing_time': end_time - start_time
        }
        indexing_results.append(result)
        
        print(f"✅ Hoàn thành: {count} vectors")
        
    except Exception as e:
        print(f"❌ Lỗi khi indexing {n_bin}bin_{h_type}_{metric}: {e}")
        import traceback
        traceback.print_exc()

print(f"\n🎉 Hoàn thành indexing cho tất cả {len(indexing_results)} cấu hình!")


🚀 Bắt đầu indexing cho 8 cấu hình...
📊 Sẽ index 23824 training samples cho mỗi cấu hình
⏳ Quá trình này có thể mất vài phút...

Indexing 1/8: 4bin_global_cosine
📂 Sử dụng collection có sẵn: rgb_hist_4bin_global_cosine (1632 vectors)
✅ Hoàn thành: 1632 vectors

Indexing 2/8: 4bin_global_l2
🔧 Tạo collection mới: rgb_hist_4bin_global_l2
   Cấu hình: n_bin=4, h_type=global, metric=l2
📊 Extracting features từ training set...


Indexing: 100%|██████████| 745/745 [04:50<00:00,  2.57it/s]


✅ Hoàn thành indexing: 23824 vectors trong 290.03s
⚡ Tốc độ: 82.1 vectors/s
✅ Hoàn thành: 23824 vectors

Indexing 3/8: 4bin_region_cosine
🔧 Tạo collection mới: rgb_hist_4bin_region_cosine
   Cấu hình: n_bin=4, h_type=region, metric=cosine
📊 Extracting features từ training set...


Indexing: 100%|██████████| 745/745 [06:18<00:00,  1.97it/s]


✅ Hoàn thành indexing: 23824 vectors trong 378.93s
⚡ Tốc độ: 62.9 vectors/s
✅ Hoàn thành: 23824 vectors

Indexing 4/8: 4bin_region_l2
🔧 Tạo collection mới: rgb_hist_4bin_region_l2
   Cấu hình: n_bin=4, h_type=region, metric=l2
📊 Extracting features từ training set...


...

✅ Hoàn thành indexing: 23824 vectors trong 185.58s
⚡ Tốc độ: 128.4 vectors/s
✅ Hoàn thành: 23824 vectors

Indexing 7/8: 8bin_region_cosine
🔧 Tạo collection mới: rgb_hist_8bin_region_cosine
   Cấu hình: n_bin=8, h_type=region, metric=cosine
📊 Extracting features từ training set...


Indexing: 100%|██████████| 745/745 [05:30<00:00,  2.25it/s]


✅ Hoàn thành indexing: 23824 vectors trong 330.44s
⚡ Tốc độ: 72.1 vectors/s
✅ Hoàn thành: 23824 vectors

Indexing 8/8: 8bin_region_l2
🔧 Tạo collection mới: rgb_hist_8bin_region_l2
   Cấu hình: n_bin=8, h_type=region, metric=l2
📊 Extracting features từ training set...


Indexing: 100%|██████████| 745/745 [05:33<00:00,  2.24it/s]

✅ Hoàn thành indexing: 23824 vectors trong 333.03s
⚡ Tốc độ: 71.5 vectors/s
✅ Hoàn thành: 23824 vectors

🎉 Hoàn thành indexing cho tất cả 8 cấu hình!


# Evaluation

In [7]:
def get_train_labels(collection):
    """Lấy labels mapping từ collection"""
    try:
        # Thử lấy từ labels map
        result = collection.get(ids=["_labels_map_"])
        if result['documents'] and len(result['documents']) > 0:
            return json.loads(result['documents'][0])
    except:
        pass
    
    # Fallback: lấy từ metadata
    all_data = collection.get(where={"type": "train"})
    labels_map = {}
    for id, metadata in zip(all_data['ids'], all_data['metadatas']):
        if 'label' in metadata:
            labels_map[id] = metadata['label']
    
    return labels_map

def evaluate_configuration_v2(n_bin, h_type, metric):
    """Evaluate một cấu hình với metrics mới"""
    collection_name = get_collection_name(n_bin, h_type, metric)
    
    try:
        collection = chroma_client.get_collection(collection_name)
        train_labels_map = get_train_labels(collection)
        
        print(f"📊 Found {len(train_labels_map)} training vectors")
        
        if len(train_labels_map) == 0:
            print("❌ Không có training data! Vui lòng chạy indexing trước.")
            return None
        
    except Exception as e:
        print(f"❌ Không tìm thấy collection {collection_name}: {e}")
        print("Vui lòng chạy indexing trước!")
        return None
    
    # Extract test features và evaluate
    feature_extractor = RGBHistogram(n_bin=n_bin, h_type=h_type, n_slice=N_SLICE)
    
    all_results = []
    test_labels = []
    tested_count = 0
    
    start_time = time()
    
    for images, labels, _ in tqdm(test_loader, desc="Testing"):
        if tested_count >= TEST_SIZE:
            break
        
        count = min(len(images), TEST_SIZE - tested_count)
        batch_images = images[:count]
        batch_labels = labels[:count]
        
        # Convert to numpy directly (RGBHistogram only works on CPU)
        # No need to move to GPU since feature extraction is CPU-only
        images_np = (batch_images.numpy().transpose(0, 2, 3, 1) * 255).astype(np.uint8)
        
        for img, label in zip(images_np, batch_labels):
            if tested_count >= TEST_SIZE:
                break
            
            # Extract test feature
            test_feature = feature_extractor(img)
            
            # Query ChromaDB
            results = collection.query(
                query_embeddings=[test_feature.tolist()],
                n_results=max(max(MAP_K_VALUES), max(RECALL_K_VALUES)),
                where={"type": "train"}
            )
            
            # Lấy labels của kết quả
            if results['ids'] and len(results['ids'][0]) > 0:
                retrieved_ids = results['ids'][0]
                retrieved_labels = [train_labels_map.get(id, -1) for id in retrieved_ids]
                all_results.append(retrieved_labels)
            else:
                all_results.append([])
            
            test_labels.append(int(label))
            tested_count += 1
    
    retrieval_time = time() - start_time
    
    # Tính metrics
    metrics_data = {}
    
    # Tính mAP với MAP_K_VALUES
    for k in MAP_K_VALUES:
        map_scores = []
        
        for retrieved_labels, gt_label in zip(all_results, test_labels):
            if len(retrieved_labels) == 0:
                map_scores.append(0.0)
                continue
            
            # Top-k results
            top_k_labels = retrieved_labels[:k]
            
            # Tính mAP
            ap = average_precision(top_k_labels, [gt_label], k)
            map_scores.append(ap)
        
        # Lưu mAP
        metrics_data[f'mAP@{k}'] = np.mean(map_scores)
    
    # Tính Recall với RECALL_K_VALUES
    for k in RECALL_K_VALUES:
        recall_scores = []
        
        for retrieved_labels, gt_label in zip(all_results, test_labels):
            if len(retrieved_labels) == 0:
                recall_scores.append(0.0)
                continue
            
            # Top-k results
            top_k_labels = retrieved_labels[:k]
            
            # Tính Recall
            relevant_in_topk = sum(1 for label in top_k_labels if label == gt_label)
            # Tổng số relevant items trong toàn bộ training set
            total_relevant = sum(1 for label in train_labels_map.values() if label == gt_label)
            
            if total_relevant > 0:
                rec = relevant_in_topk / total_relevant
            else:
                rec = 0.0
            
            recall_scores.append(rec)
        
        # Lưu Recall
        metrics_data[f'Recall@{k}'] = np.mean(recall_scores)
    
    # Tính average mAP
    avg_map = np.mean([metrics_data[f'mAP@{k}'] for k in MAP_K_VALUES])
    metrics_data['avg_mAP'] = avg_map
    
    # Hiển thị kết quả
    print("📊 Evaluation results:")
    for k in MAP_K_VALUES:
        print(f"   mAP@{k}: {metrics_data[f'mAP@{k}']:.4f}")
    for k in RECALL_K_VALUES:
        print(f"   Recall@{k}: {metrics_data[f'Recall@{k}']:.4f}")
    print(f"   Average mAP: {avg_map:.4f}")
    
    # Thêm thông tin cấu hình với average time
    metrics_data.update({
        'n_bin': n_bin,
        'h_type': h_type,
        'metric': metric,
        'config_name': f"{n_bin}bin_{h_type}_{metric}",
        'avg_retrieval_time': retrieval_time / tested_count,
        'avg_retrieval_time_ms': (retrieval_time / tested_count) * 1000,
        'tested_samples': tested_count,
        'train_samples': len(train_labels_map)
    })
    
    return metrics_data

print("📋 Hàm evaluation  đã sẵn sàng!")


📋 Hàm evaluation  đã sẵn sàng!


In [8]:
print(f"Evaluation cho {len(CONFIGS)} trường hợp...")
print(f"Test trên {TEST_SIZE} samples")

evaluation_results = []

for i, (n_bin, h_type, metric) in enumerate(CONFIGS, 1):
    print(f"\n{'='*60}")
    print(f"Evaluation {i}/{len(CONFIGS)}: {n_bin}bin_{h_type}_{metric}")
    print(f"{'='*60}")
    
    try:
        result = evaluate_configuration_v2(n_bin, h_type, metric)
        if result:
            evaluation_results.append(result)
            print(f"Complete: avg_mAP = {result['avg_mAP']:.4f}")
            print(f"Avg retrieval time: {result['avg_retrieval_time']:.4f}s ({result['avg_retrieval_time_ms']:.1f}ms)")
        else:
            print(f"Fail: {n_bin}bin_{h_type}_{metric}")
            
    except Exception as e:
        print(f"Error when evaluating {n_bin}bin_{h_type}_{metric}: {e}")
        import traceback
        traceback.print_exc()

print(f"Complete .")


Evaluation cho 8 trường hợp...
Test trên 5956 samples

Evaluation 1/8: 4bin_global_cosine
📊 Found 1632 training vectors


Testing: 100%|██████████| 187/187 [14:41<00:00,  4.72s/it]


📊 Evaluation results:
   mAP@1: 0.0155
   mAP@5: 0.0276
   mAP@10: 0.0307
   Recall@10: 0.0194
   Recall@100: 0.1201
   Recall@1000: 0.6853
   Average mAP: 0.0246
Complete: avg_mAP = 0.0246
Avg retrieval time: 0.1480s (148.0ms)

Evaluation 2/8: 4bin_global_l2
📊 Found 23824 training vectors


Testing: 100%|██████████| 187/187 [40:06<00:00, 12.87s/it] 


📊 Evaluation results:
   mAP@1: 0.0315
   mAP@5: 0.0493
   mAP@10: 0.0537
   Recall@10: 0.0024
   Recall@100: 0.0147
   Recall@1000: 0.0876
   Average mAP: 0.0448
Complete: avg_mAP = 0.0448
Avg retrieval time: 0.4041s (404.1ms)

Evaluation 3/8: 4bin_region_cosine
📊 Found 23824 training vectors


...

📊 Evaluation results:
   mAP@1: 0.0344
   mAP@5: 0.0547
   mAP@10: 0.0591
   Recall@10: 0.0027
   Recall@100: 0.0152
   Recall@1000: 0.0861
   Average mAP: 0.0494
Complete: avg_mAP = 0.0494
Avg retrieval time: 0.1686s (168.6ms)

Evaluation 7/8: 8bin_region_cosine
📊 Found 23824 training vectors


Testing: 100%|██████████| 187/187 [17:45<00:00,  5.70s/it]


📊 Evaluation results:
   mAP@1: 0.0426
   mAP@5: 0.0682
   mAP@10: 0.0732
   Recall@10: 0.0037
   Recall@100: 0.0187
   Recall@1000: 0.0859
   Average mAP: 0.0613
Complete: avg_mAP = 0.0613
Avg retrieval time: 0.1790s (179.0ms)

Evaluation 8/8: 8bin_region_l2
📊 Found 23824 training vectors


Testing: 100%|██████████| 187/187 [17:14<00:00,  5.53s/it]


📊 Evaluation results:
   mAP@1: 0.0441
   mAP@5: 0.0696
   mAP@10: 0.0741
   Recall@10: 0.0036
   Recall@100: 0.0188
   Recall@1000: 0.0862
   Average mAP: 0.0626
Complete: avg_mAP = 0.0626
Avg retrieval time: 0.1737s (173.7ms)
Complete .


In [9]:
if evaluation_results:
    print(f"\n{'='*80}")
    print(f"{'='*80}")
    
    # Hien thi ket qua theo thu tu cau hinh
    print(f"\nResults:")
    print("-" * 120)
    print(f"{'STT':<4} {'Configuration':<20} {'Avg mAP':<10} {'mAP@1':<8} {'mAP@5':<8} {'mAP@10':<8} {'R@10':<8} {'R@100':<8} {'R@1000':<8} {'Avg Time(ms)':<12}")
    print("-" * 120)
    
    for i, result in enumerate(evaluation_results, 1):
        print(f"{i:<4} {result['config_name']:<20} {result['avg_mAP']:<10.4f} "
              f"{result['mAP@1']:<8.4f} {result['mAP@5']:<8.4f} {result['mAP@10']:<8.4f} "
              f"{result['Recall@10']:<8.4f} {result['Recall@100']:<8.4f} {result['Recall@1000']:<8.4f} "
              f"{result['avg_retrieval_time_ms']:<12.1f}")
    
    # Luu ket qua
    df = pd.DataFrame(evaluation_results)
    csv_path = 'out/evaluation_results_v2.csv'
    df.to_csv(csv_path, index=False)
    print(f"\nKet qua chi tiet luu tai: {csv_path}")
    
    # Thong ke tong quan
    print(f"- Số trường hợp đã test: {len(evaluation_results)}")
    print(f"- Test samples: {evaluation_results[0]['tested_samples']}")
    print(f"- Training samples: {evaluation_results[0]['train_samples']}")
    
    avg_times = [r['avg_retrieval_time_ms'] for r in evaluation_results]
    avg_maps = [r['avg_mAP'] for r in evaluation_results]
    
    print(f"- Avg time: {np.mean(avg_times):.1f}ms (min: {np.min(avg_times):.1f}ms, max: {np.max(avg_times):.1f}ms)")
else:
    print("Empty !!")
    print("Hãy indexing trước!.")

# GPU memory cleanup
if device.type == "cuda":
    torch.cuda.empty_cache()
    print("\nGPU memory cleaned up")




Results:
------------------------------------------------------------------------------------------------------------------------
STT  Configuration        Avg mAP    mAP@1    mAP@5    mAP@10   R@10     R@100    R@1000   Avg Time(ms)
------------------------------------------------------------------------------------------------------------------------
1    4bin_global_cosine   0.0246     0.0155   0.0276   0.0307   0.0194   0.1201   0.6853   148.0       
2    4bin_global_l2       0.0448     0.0315   0.0493   0.0537   0.0024   0.0147   0.0876   404.1       
3    4bin_region_cosine   0.0674     0.0468   0.0753   0.0801   0.0042   0.0203   0.0930   162.1       
4    4bin_region_l2       0.0673     0.0473   0.0745   0.0800   0.0043   0.0204   0.0928   161.8       
5    8bin_global_cosine   0.0494     0.0345   0.0547   0.0591   0.0027   0.0152   0.0860   163.5       
6    8bin_global_l2       0.0494     0.0344   0.0547   0.0591   0.0027   0.0152   0.0861   168.6       
7    8bin_region_co